In [10]:
import pypdf
import chromadb
import urllib3
import accelerate
import sentence_transformers
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer

In [11]:
%env TOKENIZERS_PARALLELISM=True
%env TF_CPP_MIN_LOG_LEVEL=3

env: TOKENIZERS_PARALLELISM=True
env: TF_CPP_MIN_LOG_LEVEL=3


## Extracting text data from PDF Files

In [12]:
# Create an object to load PDF file
loader = PyPDFLoader('../data/ArtigoDSA1.pdf')

type(loader)

langchain_community.document_loaders.pdf.PyPDFLoader

In [13]:
# Load the file PDF
pages = loader.load()

pages

[Document(metadata={'source': '../data/ArtigoDSA1.pdf', 'page': 0, 'page_label': '1'}, page_content='A Habilidade Mais Importante na Era da Inteligência Artificial \n \nA pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo, já que \ntudo, desde reuniões até consultas médicas, ficou online. Isso pode soar como algo super \npositivo.  \nPara dezenas de milhões de trabalhadores, não. \nEles talvez não tenham as habilidades necessárias para competir nesse novo mundo. Eles são os \ncontadores, os digitadores, os secretários executivos, procurando trabalho em uma nova \neconomia na qual as pessoas contratadas têm títulos como “Engenheiro de Nuv em” ou “Hacker \nde Crescimento” em seus currículos. Sem um esforço concentrado para retreiná-los, descobriram \nos pesquisadores da RAND Europe, eles provavelmente serão deixados para trás. \nE não apenas eles. O custo dessa crescente lacuna de habilidades será medido em trilhões de \ndólares e recairá mais fortemente em 

In [14]:
page = pages[2]

print("Page content: ", page.page_content[0:500])

Page content:  Não importa sua área, seu mercado, sua graduação, sua idade ou seu gênero. O mundo está 
passando por uma profunda transformação digital e os empregos como conhecemos estão sendo 
reinventados. Aqueles que não acompanharem essa evolução natural ficarão para trás, como 
tantas vezes vimos na história humana. Aprenda o máximo que puder, sobre diferentes temas, 
desde habilidades interpessoais até habilidades técnicas. O único limite sobre o que você pode 
aprender é o que você impõe a si mesmo. 
“


In [15]:
print("Metadata:", page.metadata)

Metadata: {'source': '../data/ArtigoDSA1.pdf', 'page': 2, 'page_label': '3'}


## Splitting Text Data in Chunks

**chunk_size = 1000**: Specifies that each resulting chunk of text will have a maximum of 1000 characters.

**chunk_overlap = 20**: Indicates that each chunk will have 20 characters of overlap with the next chunk. This means that the last 20 characters of a chunk will be repeated at the beginning of the next chunk.

What it is for:

This approach is useful in several situations where large texts need to be processed or analyzed, such as:

- Input for language models: Many LLMs have a limit of tokens that they can process in a single iteration. Dividing the text into smaller chunks ensures that the text is sent within the allowed limit.

- Data analysis and indexing: It is common in search engines and data processing pipelines, where dividing the text into smaller chunks makes it easier to index and retrieve information.

- Context maintenance: When processing involves long documents, this technique allows you to deal with them more efficiently, by dividing the parts without losing logic or cohesion.

In [16]:
# Create the chunk text separator
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 20)

In [17]:
# Applying the object and extracting the chunks (documents)
docs = splitter.split_documents(pages)

print("Total of Chunks (Documents):", len(docs))

print("Last Chunk Content (Document):", docs[6])

Total of Chunks (Documents): 7
Last Chunk Content (Document): page_content='Não importa sua área, seu mercado, sua graduação, sua idade ou seu gênero. O mundo está 
passando por uma profunda transformação digital e os empregos como conhecemos estão sendo 
reinventados. Aqueles que não acompanharem essa evolução natural ficarão para trás, como 
tantas vezes vimos na história humana. Aprenda o máximo que puder, sobre diferentes temas, 
desde habilidades interpessoais até habilidades técnicas. O único limite sobre o que você pode 
aprender é o que você impõe a si mesmo. 
“Seja Bom em Aprender”. Mantenha-se em modo constante de aprendizado. 
Equipe DSA 
www.datascienceacademy.com.br' metadata={'source': '../data/ArtigoDSA1.pdf', 'page': 2, 'page_label': '3'}


---

## Loading Text Data Vectors into the Vector Database

The code implements a semantic search system using a vector database (vectordb) to identify the most relevant points in relation to a question, based on the semantic similarity between the documents and the provided question.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

https://www.trychroma.com/

In [18]:
# Create the vector database
vectordb = Chroma.from_documents(documents = docs,
                                 embedding = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2"),
                                 persist_directory = "vectordb/chroma/")

**Chroma.from_documents(documents=docs)**: Creates a vector database using the provided documents (stored in the docs variable). These documents can be texts, articles, or any type of textual data that you want to index.

**HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")**: Uses the Hugging Face all-MiniLM-L6-v2 semantic embedding model to transform texts into numeric vectors. Embeddings are mathematical representations of texts that capture their semantic meaning.

**persist_directory="dsavectordb/chroma/"**: Specifies the directory where the vector database will be saved (persisted), so that it can be reused in future sessions without having to reprocess the documents.

In [19]:
# Total collections in vector db
vectordb._collection.count()

21

## Testing Vector Search Parameters

In [20]:
# Define a question
question = "Has the COVID-19 pandemic accelerated the pace of digital development around the world?"

In [21]:
# Perform the vector search
relevant_points = vectordb.max_marginal_relevance_search(question, k = 2, fetch_k = 3)
print(relevant_points)

[Document(metadata={'page': 0, 'page_label': '1', 'source': '../data/ArtigoDSA1.pdf'}, page_content='A Habilidade Mais Importante na Era da Inteligência Artificial \n \nA pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo, já que \ntudo, desde reuniões até consultas médicas, ficou online. Isso pode soar como algo super \npositivo.  \nPara dezenas de milhões de trabalhadores, não. \nEles talvez não tenham as habilidades necessárias para competir nesse novo mundo. Eles são os \ncontadores, os digitadores, os secretários executivos, procurando trabalho em uma nova \neconomia na qual as pessoas contratadas têm títulos como “Engenheiro de Nuv em” ou “Hacker \nde Crescimento” em seus currículos. Sem um esforço concentrado para retreiná-los, descobriram \nos pesquisadores da RAND Europe, eles provavelmente serão deixados para trás. \nE não apenas eles. O custo dessa crescente lacuna de habilidades será medido em trilhões de \ndólares e recairá mais fortemente em 

**max_marginal_relevance_search()**: Performs a search in the vector database based on maximal marginal relevance (MMR). This technique is used to find documents that are relevant to the given question, reducing redundancy in the answers. Instead of returning documents that are very similar to each other, it ensures diversity in the answers while maintaining relevance. Read the pdf manual in Chapter 16 for more details.

Parameters:

**question**: The natural text question used to calculate the semantic similarity with the documents in the vector database.

**k=2**: Defines the number of final documents that will be returned as the most relevant.

**fetch_k=3**: Specifies that the algorithm should initially search for the 3 most relevant documents and then apply the MMR technique to select the 2 most diverse and relevant.

In [22]:
print(relevant_points[0])

page_content='A Habilidade Mais Importante na Era da Inteligência Artificial 
 
A pandemia do COVID-19 acelerou o ritmo do desenvolvimento digital em todo o mundo, já que 
tudo, desde reuniões até consultas médicas, ficou online. Isso pode soar como algo super 
positivo.  
Para dezenas de milhões de trabalhadores, não. 
Eles talvez não tenham as habilidades necessárias para competir nesse novo mundo. Eles são os 
contadores, os digitadores, os secretários executivos, procurando trabalho em uma nova 
economia na qual as pessoas contratadas têm títulos como “Engenheiro de Nuv em” ou “Hacker 
de Crescimento” em seus currículos. Sem um esforço concentrado para retreiná-los, descobriram 
os pesquisadores da RAND Europe, eles provavelmente serão deixados para trás. 
E não apenas eles. O custo dessa crescente lacuna de habilidades será medido em trilhões de 
dólares e recairá mais fortemente em lugares que não possuem infraestrutura digital confiável,' metadata={'page': 0, 'page_label': '1', 

## Defining LLM

https://huggingface.co/Qwen/Qwen2.5-1.5B-Instruct

In [26]:
# Set the name of the LLM as it appears in the HF
llm_model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [24]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(llm_model_name, 
                                             torch_dtype = "auto", 
                                             device_map = "auto")

c:\Users\krupc\Downloads\Projects\mlops\mem-rag\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\krupc\.cache\huggingface\hub\models--Qwen--Qwen2.5-1.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [27]:
# Load the tokenizer from the model
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)

---

## Setting the Context

In [29]:
# Defining the question
question = "Has the COVID-19 pandemic accelerated the pace of digital development around the world?"

# Extract the context of the question (i.e. perform vector search)
context = vectordb.max_marginal_relevance_search(question, k = 2, fetch_k = 3)

## Setting the Prompt

In [30]:
# Create the prompt
prompt = f"""
You are an expert assistant. You use the context provided as your complementary knowledge base to answer the question.
context = {context}
question = {question}
answer =
"""

In [31]:
# Create the list of system and user messages
messages = [
{"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are an expert assistant."},
{"role": "user", "content": prompt}
]